# Library


In [8]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [3]:
drive = 'D'

In [4]:
ods_fold = f'{drive}:/data/big2/ODS'
big2_fold = f'{drive}:/data/big2'
analysis_fold = f'{drive}:/data/big2/BD3/an'

# Load

## 등록정보(STD_CEG_CAR_MIG) 4등급만

In [5]:
# # insider db
# wb = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')

In [9]:
# exasol db
# 8.6s
wb = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4';")
# car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD FROM STD_CEG_CAR_MIG;")

In [10]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
}

In [11]:
carr = car.rename(columns=car_ch_col)

In [12]:
carr.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [13]:
# 3.8s
src = wb.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY FROM STD_CEG_CAR_SRC_MIG;")

In [14]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
}

In [15]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식'], dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [16]:
# 1.3s
code = wb.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46333 entries, 0 to 46332
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       46333 non-null  int64 
 1   STDG_CTPV_NM  46333 non-null  object
 2   STDG_SGG_NM   46310 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [17]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_NM', 'STDG_SGG_NM'], dtype='object')

In [18]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}

In [19]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

## 노후차 조기폐차 관리정보(수도권)(STD_DLM_TB_ERP_EARLY_ERASE_AEA)

In [20]:
# 2.4s
aea = wb.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN FROM STD_DLM_TB_ERP_EARLY_ERASE_AEA;")
aea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804082 entries, 0 to 804081
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   VIN                  804082 non-null  object
 1   ELPDSRC_STTS_CD      804082 non-null  object
 2   ELPDSRC_LST_APRV_YN  683874 non-null  object
dtypes: object(3)
memory usage: 18.4+ MB


In [21]:
aea.columns

Index(['VIN', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN'], dtype='object')

In [22]:
aea_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN',  
}

In [23]:
aear = aea.rename(columns=aea_ch_col)
aear.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN'], dtype='object')

## 노후차 조기폐차 관리정보(수도권외)(STD_DLM_TB_ERP_EARLY_ERASE_LGV)

In [24]:
# 2.8s
lgv = wb.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN FROM STD_DLM_TB_ERP_EARLY_ERASE_AEA;")
lgv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804082 entries, 0 to 804081
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   VIN                  804082 non-null  object
 1   ELPDSRC_STTS_CD      804082 non-null  object
 2   ELPDSRC_LST_APRV_YN  683874 non-null  object
dtypes: object(3)
memory usage: 18.4+ MB


In [25]:
lgv.columns

Index(['VIN', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN'], dtype='object')

In [26]:
lgv_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN',  
}

In [27]:
lgvr = lgv.rename(columns=lgv_ch_col)
lgvr.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN'], dtype='object')

# 전처리

## 중복 차대번호 제거

In [29]:
carr.shape, len(carr['차대번호'].unique())

((1898814, 10), 1898763)

In [30]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(1898763, 10)

## 등록정보 말소 제거

In [31]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(1515736, 10)

## 차종 코드 변환

In [32]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carm['차종'] = carm['차종'].replace(cd_dict)
carm['차종'].unique()

array(['화물', '승용', '승합', '특수', nan], dtype=object)

## 용도 코드 변환

In [33]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carm['용도'] = carm['용도'].replace(purps_dict)
carm['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 연료 코드 변환

In [34]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '전기', '알코올', '등유', 'CNG 하이브리드', '태양열', '경유 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

## 차종유형 코드 변환

In [35]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [36]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(1515736, 16)

## 지역 정보 부착

In [38]:
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

In [40]:
csc = cs.merge(coder, on='법정동코드', how='left')
csc.shape

(1515736, 18)

In [41]:
csc['시도'].isnull().sum()

1

In [42]:
# 경기도 양주시
csc.loc[csc['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시']

In [43]:
csc['시도'].isnull().sum()

0

## 조기폐차 정보들 병합

In [44]:
elp = pd.concat([aear, lgvr], ignore_index=True)
elp.shape

(1608164, 3)

In [45]:
elp.shape, len(elp['차대번호'].unique())

((1608164, 3), 758734)

In [49]:
elpm = elp.sort_values('조기폐차최종승인YN', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
elpm.shape

(758734, 3)

In [51]:
elpm = elpm[elpm['조기폐차최종승인YN'] == 'Y'].reset_index(drop=True)
elpm.shape

(683814, 3)

## 조기폐차 정보 추가

In [52]:
csc.shape

(1515736, 18)

In [53]:
df = csc.merge(elpm, on='차대번호', how='left')
df.shape

(1515736, 20)

In [55]:
df['연료'].value_counts(dropna=False)

연료
경유             1156239
휘발유             333206
LPG(액화석유가스)      26179
NaN                 71
기타연료                39
CNG(압축천연가스)          1
알코올                  1
Name: count, dtype: int64

In [56]:
df = df[df['연료'] == '경유'].reset_index(drop=True)

(1156239, 20)

## STD_BD_GRD4_ELPDSRC_CURSTT

In [57]:
df.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '시도',
       '시군구', '조기폐차상태코드', '조기폐차최종승인YN'],
      dtype='object')

In [58]:
STD_BD_GRD4_ELPDSRC_CURSTT = df[[
    '차대번호', 
    '법정동코드', 
    '차종', 
    '용도', 
    '연료', 
    '차종유형', 
    '시도',
    '시군구', 
    '조기폐차상태코드', 
    '조기폐차최종승인YN',
]]

In [59]:
today_date = datetime.today().strftime("%Y%m")
today_date

'202306'

In [60]:
STD_BD_GRD4_ELPDSRC_CURSTT['기준연월'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_9792\3938275755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STD_BD_GRD4_ELPDSRC_CURSTT['기준연월'] = today_date


In [61]:
STD_BD_GRD4_ELPDSRC_CURSTT.columns

Index(['차대번호', '법정동코드', '차종', '용도', '연료', '차종유형', '시도', '시군구', '조기폐차상태코드',
       '조기폐차최종승인YN', '기준연월'],
      dtype='object')

In [ ]:
STD_BD_GRD4_ELPDSRC_CURSTT = STD_BD_GRD4_ELPDSRC_CURSTT[[
    '기준연월', 
    '차대번호', 
    '법정동코드', 
    '차종', 
    '용도', 
    '연료', 
    '차종유형', 
    '시도', 
    '시군구', 
    '조기폐차상태코드',
    '조기폐차최종승인YN', 
]]

In [63]:
chc_dict = {
    '기준연월':'CRTR_YM', 
    '차대번호':'VIN', 
    '법정동코드':'STDG_CD', 
    '차종':'VHCTY_CD', 
    '용도':'PURPS_CD2', 
    '연료':'FUEL_CD', 
    '차종유형':'VHCTY_TY', 
    '시도':'CTPV', 
    '시군구':'SGG', 
    '조기폐차상태코드':'ELPDSRC_STTS_CD',
    '조기폐차최종승인YN':'ELPDSRC_LST_APRV_YN', 
}

In [65]:
STD_BD_GRD4_ELPDSRC_CURSTT = STD_BD_GRD4_ELPDSRC_CURSTT.rename(columns=chc_dict)
STD_BD_GRD4_ELPDSRC_CURSTT.columns

Index(['VIN', 'STDG_CD', 'VHCTY_CD', 'PURPS_CD2', 'FUEL_CD', 'VHCTY_TY',
       'CTPV', 'SGG', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN', 'CRTR_YM'],
      dtype='object')

# code end